# Gradients: Adjoint differentiation (500 points)
In the last two problems, we dove into the parameter-shift rule. It's an elegant, simple, and *extremely* useful way to differentiate quantum circuits because of its hardware compatibility. There are other differentiation methods that exist and are very efficient, but are *not* hardware compatible in general. We'll describe a way to perform one of those methods in this challenge: adjoint differentiation.

Consider a circuit given by a unitary $U(\vec{\theta}) = \prod_{i=1}^n U_i(\theta_i)$, where $\vec{\theta}$ are real differentiable parameters, whose output is an expectation value with respect to an observable $A$:

$$f(\vec{\theta}) = \langle 0 | U^{\dagger}(\vec{\theta}) A U(\vec{\theta}) | 0 \rangle$$

Let's calculate, say, the derivative of $f$ with respect to the parameter $\theta_i$ from a naive perspective. There are two instances in $f(\vec{\theta})$ that involve dependence on $\theta_i$: one from $U_i(\theta_i)$ and one from $U_i^{\dagger}(\theta_i)$. Thus, we will have to use the product rule as follows.

$$\begin{aligned}
    \frac{\partial}{\partial \theta_i} f(\vec{\theta})
    = &\langle 0 | \left( \prod_{j=1}^n \frac{\partial}{\partial \theta_i} U_j^{\dagger}(\theta_j) \right) A U(\vec{\theta}) | 0 \rangle\\
      &+ \langle 0 | U^{\dagger}(\vec{\theta}) A \left( \prod_{j=1}^n \frac{\partial}{\partial \theta_i} U_j(\theta_j) \right) | 0 \rangle\\
    = &\langle 0 | U_1^{\dagger} \ldots \frac{\partial}{\partial \theta_i} U_i^{\dagger} \ldots U_n^{\dagger} A U(\vec{\theta}) | 0 \rangle\\
      &+ \langle 0 | U^{\dagger}(\vec{\theta}) A U_n \ldots \frac{\partial}{\partial \theta_i} U_i \ldots U_1 | 0 \rangle\\
\end{aligned}$$

Note that we are assuming that each parameterized gate is parameterized by only *one* parameter, and we've eliminated the $\theta_i$ dependence in each gate $U_i(\theta_i)$ for legibility.

One way to evaluate this expression, albeit a very inefficient way, is to simulate four quantum circuits that all return `qml.state()` with some crucial assumptions:

1. Each gate $U_i$ must be of the form $U(\theta) = \exp(ic\theta G)$, where $c$ is a real number (constant) and $G$ is unitary ($G$ is often called the "generator" of $U$).
2. The observable $A$ must also be unitary for our purposes.

Can you guess why these assumptions have to be made in order to implement this expression using quantum circuits?

Nevertheless, let's stick with these assumptions. Conveniently,

$$\frac{\partial}{\partial \theta_i} U_i = i c G U_i$$

Therefore,

$$\begin{aligned}
    \frac{\partial}{\partial \theta_i} f(\vec{\theta})
    = &\langle 0 | U_1^{\dagger} \ldots \left( -i c G^{\dagger} U_i^{\dagger} \right) \ldots U_n^{\dagger} A U(\vec{\theta}) | 0 \rangle\\
      &+ \langle 0 | U^{\dagger}(\vec{\theta}) A U)n \ldots (i c G U_i) \ldots U_1 | 0 \rangle\\
    = &c \left[\begin{aligned}
        &\langle 0 | U_1^{\dagger} \ldots \left( -i G^{\dagger} U_i^{\dagger} \right) \ldots U_n^{\dagger} A U(\vec{\theta}) | 0 \rangle\\
        &+ \langle 0 | U^{\dagger}(\vec{\theta)} A U_n \ldots (i G U_i) \ldots U_1 | 0 \rangle\\
    \end{aligned}\right]
\end{aligned}$$

Each term in the expression above can be boiled down to the following:

$$\frac{\partial}{\partial \theta_i} f(\vec{\theta}) = c[\langle \mathrm{bra}_1 | \mathrm{ket}_1 \rangle + \langle \mathrm{bra}_2 | \mathrm{ket}_2 \rangle]$$

where the freedom for how to define each bra and ket is left to you! Surely, if four PennyLane circuits are made that create each bra and ket, the derivative can be calculated with two inner products added together and multiplied by a coefficient $c$...

For the keen readers: do you actually need four circuits 🧐?

## Challenge code
In the code below, you are given a few functions:
- `generator_info`: This function returns the generator $G$ and coefficient $c$ given an operator $U$.
- `derivative`: This is where you will calculate the derivative of $f(\vec{\theta})$ with respect to one parameter. Within this function, there are several circuits that you must complete:
    - `circuit_bra1`: calculates $\langle \mathrm{bra}_1 |$ (or $| \mathrm{bra}_1 \rangle$ depending on how you want to do things!)
    - `circuit_ket1`: calculates $| \mathrm{ket}_1 \rangle$
    - `circuit_bra2`: calculates $\langle \mathrm{bra}_2 |$ (or $| \mathrm{bra}_2 \rangle$ depending on how you want to do things!)
    - `circuit_ket2`: calculates $| \mathrm{ket}_2 \rangle$

### Inputs
As input to this problem, you are given:
- `op_order` (`list(int)`): Each gate $U_i$ is one of the Pauli rotation gates: `qml.RX`, `qml.RY`, or `qml.RZ`. `op_order` defines the sequence of Pauli rotation gates that defines the circuit $U(\vec{\theta})$.
- `params` (`list(float)`): This is a list of parameters corresponding to $\vec{\theta}$.
- `diff_idx`: This is an integer that relates to the parameter that will be differentiated with respect to.
- `wires` (`list(int)`): This is a list of wires that each rotation gate will be applied to.
- `measured_wire` (`int`): The wire that the observable $A$ will be applied to.

In our case, $A = Z$, the Pauli Z operator.

### Outputs
This code will output the derivative, a `float`, of the circuit with respect to the given parameter.

If your solution matches the correct one within the given tolerance specified in `check` (in this case it's a `1e-4` relative error tolerance), the output will be `"Correct!"`. Otherwise, you will receive a `"Wrong answer"` prompt.

Good luck!

### Code

In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
def generator_info(operator):
    """Provides the generator of a given operator.

    Args:
        operator (qml.ops): A PennyLane operator

    Returns:
        (qml.ops): The generator of the operator.
        (float): The coefficient of the generator.
    """
    gen = qml.generator(operator, format="observable")
    return gen.ops[0], gen.coeffs[0]


In [3]:
def derivative(op_order, params, diff_idx, wires, measured_wire):
    """A function that calculates the derivative of a circuit w.r.t. one parameter.

    NOTE: you cannot use qml.grad in this function.

    Args:
        op_order (list(int)):
            This is a list of integers that defines the circuit in question.
            The entries of this list correspond to dictionary keys to op_dict.
            For example, [1,0,2] means that the circuit in question contains
            an RY gate, an RX gate, and an RZ gate in that order.

        params (np.array(float)):
            The parameters that define the gates in the circuit. In this case,
            they're all rotation angles.

        diff_idx (int):
            The index of the gate in the circuit that is to be differentiated
            with respect to. For instance, if diff_idx = 2, then the derivative
            of the third gate in the circuit will be calculated.

        wires (list(int)):
            A list of wires that each gate in the circuit will be applied to.

        measured_wire (int):
            The expectation value that needs to be calculated is with respect
            to the Pauli Z operator. measured_wire defines what wire we're
            measuring on.

    Returns:
        float: The derivative evaluated at the given parameters.
    """
    op_dict = {0: qml.RX, 1: qml.RY, 2: qml.RZ}
    dev = qml.device("default.qubit", wires=2)

    obs = qml.PauliZ(measured_wire)
    operator = op_dict[op_order[diff_idx]](params[diff_idx], wires[diff_idx])
    gen, coeff = generator_info(operator)

    @qml.qnode(dev)
    def circuit_bra1():

        # dU(theta) |0> / -ic
        for i in range(diff_idx):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(gen)
        for i in range(diff_idx, len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])

        return qml.state()

    @qml.qnode(dev)
    def circuit_ket1():

        # A U(theta) |0>
        for i in range(len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(obs)
        
        return qml.state()

    @qml.qnode(dev)
    def circuit_bra2():

        # U(theta) |0>
        for i in range(len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])

        return qml.state()

    @qml.qnode(dev)
    def circuit_ket2():

        # A dU(theta) |0> / -ic
        for i in range(diff_idx):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(gen)
        for i in range(diff_idx, len(op_order)):
            op_dict[op_order[i]](params[i], wires[i])
        qml.apply(obs)

        return qml.state()

    bra1 = circuit_bra1()
    ket1 = circuit_ket1()
    bra2 = circuit_bra2()
    ket2 = circuit_ket2()

    return  np.real(1j* coeff * (-np.dot(bra1.conjugate(),ket1) 
                                 +np.dot(bra2.conjugate(),ket2)))


In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    op_order, params, diff_idx, wires, measured_wire = json.loads(test_case_input)
    params = np.array(params, requires_grad=True)
    der = derivative(op_order, params, diff_idx, wires, measured_wire)
    return str(der)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your derivative isn't quite right!"


In [5]:
test_cases = [['[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]', '-0.2840528']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,0,2,1,0,1], [1.23, 4.56, 7.89, 1.23, 4.56, 7.89], 0, [1, 0, 1, 1, 1, 0], 1]'...
Correct!
